In [1]:
import argparse
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os
from llava.conversation import conv_templates, SeparatorStyle
from llava.utils import disable_torch_init
from transformers import CLIPVisionModel, CLIPImageProcessor, StoppingCriteria
from llava.model import *
from llava.model.utils import KeywordsStoppingCriteria

from PIL import Image

import os
from io import BytesIO

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2 from transformers import AutoTokenizer, AutoModelForCausalLM                                │
│    3 import torch                                                                                │
│    4 import os                                                                                   │
│ ❱  5 from llava.conversation import conv_templates, SeparatorStyle                               │
│    6 from llava.utils import disable_torch_init                                                  │
│    7 from transformers import CLIPVisionModel, CLIPImageProcessor, StoppingCriteria              │
│    8 from llava.model import *                                                                   │
│                                                                                                  │
│ /workspace/pre-trained-weights/LLaVA/llava/__init__.py:1 in <module>                             │
│                                                                                                  │
│ ❱ 1 from .model import LlavaLlamaForCausalLM                                                     │
│   2                                                                                              │
│                                                                                                  │
│ /workspace/pre-trained-weights/LLaVA/llava/model/__init__.py:1 in <module>                       │
│                                                                                                  │
│ ❱ 1 from .llava import LlavaLlamaForCausalLM, LlavaConfig                                        │
│   2 from .llava_mpt import LlavaMPTForCausalLM, LlavaMPTConfig                                   │
│   3                                                                                              │
│                                                                                                  │
│ /workspace/pre-trained-weights/LLaVA/llava/model/llava.py:23 in <module>                         │
│                                                                                                  │
│    20 import torch.nn.functional as F                                                            │
│    21 from torch.nn import CrossEntropyLoss                                                      │
│    22                                                                                            │
│ ❱  23 from transformers import AutoConfig, AutoModelForCausalLM, \                               │
│    24 │   │   │   │   │   │    LlamaConfig, LlamaModel, LlamaForCausalLM, \                      │
│    25 │   │   │   │   │   │    CLIPVisionModel, CLIPImageProcessor                               │
│    26                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ImportError: cannot import name 'LlamaConfig' from 'transformers' 
(/home/user/.local/lib/python3.10/site-packages/transformers/__init__.py)

In [2]:
DEFAULT_IMAGE_TOKEN = "<image>"
DEFAULT_IMAGE_PATCH_TOKEN = "<im_patch>"
DEFAULT_IM_START_TOKEN = "<im_start>"
DEFAULT_IM_END_TOKEN = "<im_end>"

In [3]:
disable_torch_init()
model_name  = os.path.expanduser("/workspace/pre-trained-weights/LLaVA/LLaVA-13B-v0")
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
if "mpt" in model_name.lower():
   model = LlavaMPTForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True, torch_dtype=torch.float16, use_cache=True).cuda()
else:
   model = LlavaLlamaForCausalLM.from_pretrained(model_name, low_cpu_mem_usage=True, torch_dtype=torch.float16, use_cache=True).cuda()
image_processor = CLIPImageProcessor.from_pretrained(model.config.mm_vision_tower, torch_dtype=torch.float16)

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.3.self_attn.out_proj.weight', 'text_model.encoder.layers.0.self_attn.v_proj.bias', 'text_model.encoder.layers.1.self_attn.out_proj.weight', 'text_model.encoder.layers.10.self_attn.out_proj.weight', 'text_model.encoder.layers.11.mlp.fc1.bias', 'text_model.encoder.layers.8.mlp.fc2.weight', 'text_model.encoder.layers.2.self_attn.v_proj.weight', 'text_model.encoder.layers.7.mlp.fc1.bias', 'text_model.encoder.layers.8.self_attn.k_proj.bias', 'text_model.encoder.layers.11.self_attn.q_proj.weight', 'text_model.encoder.layers.11.self_attn.k_proj.weight', 'text_model.encoder.layers.5.layer_norm2.bias', 'text_model.encoder.layers.5.self_attn.k_proj.weight', 'text_model.encoder.layers.2.self_attn.k_proj.bias', 'text_model.encoder.layers.3.self_attn.v_proj.bias', 'text_model.encoder.layers.5.self_attn.v_proj.weight', 'text_model.encoder.layers.2.mlp.fc1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
mm_use_im_start_end = getattr(model.config, "mm_use_im_start_end", False)
tokenizer.add_tokens([DEFAULT_IMAGE_PATCH_TOKEN], special_tokens=True)
if mm_use_im_start_end:
   tokenizer.add_tokens([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN], special_tokens=True)

In [6]:
mm_use_im_start_end

True

In [11]:
vision_tower = model.get_model().vision_tower[0]
if vision_tower.device.type == 'meta':
   vision_tower = CLIPVisionModel.from_pretrained(vision_tower.config._name_or_path, torch_dtype=torch.float16, low_cpu_mem_usage=True).cuda()
   model.get_model().vision_tower[0] = vision_tower
else:
   vision_tower.to(device='cuda', dtype=torch.float16)
vision_config = vision_tower.config
vision_config.im_patch_token = tokenizer.convert_tokens_to_ids([DEFAULT_IMAGE_PATCH_TOKEN])[0]
vision_config.use_im_start_end = mm_use_im_start_end
if mm_use_im_start_end:
   vision_config.im_start_token, vision_config.im_end_token = tokenizer.convert_tokens_to_ids([DEFAULT_IM_START_TOKEN, DEFAULT_IM_END_TOKEN])
image_token_len = (vision_config.image_size // vision_config.patch_size) ** 2

Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPVisionModel: ['text_model.encoder.layers.0.layer_norm1.bias', 'text_model.encoder.layers.9.layer_norm2.weight', 'text_model.encoder.layers.6.self_attn.v_proj.bias', 'text_model.encoder.layers.11.mlp.fc2.weight', 'text_model.encoder.layers.7.self_attn.v_proj.bias', 'text_model.encoder.layers.8.layer_norm2.weight', 'text_model.encoder.layers.3.self_attn.out_proj.bias', 'text_model.encoder.layers.5.mlp.fc2.weight', 'text_model.encoder.layers.3.mlp.fc2.bias', 'text_model.encoder.layers.8.self_attn.out_proj.weight', 'text_model.encoder.layers.0.mlp.fc1.bias', 'text_model.encoder.layers.3.layer_norm2.bias', 'text_model.encoder.layers.6.self_attn.out_proj.bias', 'text_model.encoder.layers.4.self_attn.v_proj.weight', 'text_model.encoder.layers.2.self_attn.q_proj.weight', 'text_model.encoder.layers.7.mlp.fc1.bias', 'text_model.encoder.layers.11.layer_norm1.bias', 'text_model.encoder.layers.

In [ ]:
image_file = '../grounder_output/rgb/005.png'
image = Image.open(image_file).convert('RGB')

In [12]:
qs = "Describe items on the table in full sentences"

if mm_use_im_start_end:
   qs = qs + '\n' + DEFAULT_IM_START_TOKEN + DEFAULT_IMAGE_PATCH_TOKEN * image_token_len + DEFAULT_IM_END_TOKEN
else:
   qs = qs + '\n' + DEFAULT_IMAGE_PATCH_TOKEN * image_token_len

conv_mode = "multimodal"

conv = conv_templates[conv_mode].copy()
conv.append_message(conv.roles[0], qs)
conv.append_message(conv.roles[1], None)

In [13]:
prompt = conv.get_prompt()
inputs = tokenizer([prompt])

In [14]:
image_tensor = image_processor.preprocess(image, return_tensors='pt')['pixel_values'][0]

input_ids = torch.as_tensor(inputs.input_ids).cuda()
stop_str = conv.sep if conv.sep_style != SeparatorStyle.TWO else conv.sep2
keywords = [stop_str]
stopping_criteria = KeywordsStoppingCriteria(keywords, tokenizer, input_ids)

with torch.inference_mode():
   output_ids = model.generate(
      input_ids,
      images=image_tensor.unsqueeze(0).half().cuda(),
      do_sample=True,
      temperature=0.2,
      max_new_tokens=1024,
      stopping_criteria=[stopping_criteria])

/home/user/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1211: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/home/user/.local/lib/python3.10/site-packages/transformers/generation/utils.py:2531: UserWarning: Specified kernel cache directory is not writable! This disables kernel caching. Specified directory is /home/user/.cache/torch/kernels. This warning will appear only once per process. (Triggered internally at ../aten/src/ATen/native/cuda/jit_utils.cpp:1461.)
  next_tokens.tile(eos_token_id_tensor.shape[0], 1).ne(eos_token_id_tensor.unsqueeze(1)).prod(dim=0)


In [15]:
input_token_len = input_ids.shape[1]
n_diff_input_output = (input_ids != output_ids[:, :input_token_len]).sum().item()
if n_diff_input_output > 0:
   print(f'[Warning] {n_diff_input_output} output_ids are not the same as the input_ids')
outputs = tokenizer.batch_decode(output_ids[:, input_token_len:], skip_special_tokens=True)[0]
outputs = outputs.strip()

In [16]:
if outputs.endswith(stop_str):
   outputs = outputs[:-len(stop_str)]
outputs = outputs.strip()
print(outputs)

The image shows a table in a kitchen setting with various items on it. There is a plate of food, a fork, a knife, and a spoon. Additionally, there are three bottles placed on the table. A chair is positioned next to the table, presumably for someone to sit and enjoy their meal.
